## making sure of dependency compatilibility (tf - tf.prob - tf.keras)

In [3]:
import tensorflow as tf
import tensorflow_probability as tfp

print("TensorFlow version:", tf.__version__)  # Should print 2.17.0
print("TFP version:", tfp.__version__)        # Should print 0.24.0

# Test TF-Keras integration
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
])

print("Model initialized successfully.")

TensorFlow version: 2.17.0
TFP version: 0.24.0
Model initialized successfully.


In [14]:
from uqX.modeling.uqXModel import uqXModel

(X_train, y_train), (X_val, y_val) = tf.keras.datasets.mnist.load_data()
X_train = X_train.reshape(-1, 784) / 255.0
X_val = X_val.reshape(-1, 784) / 255.0

data = (X_train, y_train, X_val, y_val)
X_train
uq_model = uqXModel()

uq_model.plot_density(X_train)
# uq_model.train_model(data, uq_method="bayes_by_backprop")

ValueError: Found array with dim 3. KernelDensity expected <= 2.